In [1]:
import sqlite3
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

O Banco de Dados em formato sqlite possuí a tabela SO_QUESTION que contém as colunas: 
<img src="imagens/BD.png" alt="drawing" width="400"/>

As informações de cada tabela são:

**SO_QUESTION** - *Stack Overflow Questions*:
- ID: identificador da questão na base de dados
- SO_ID: indentificador da questão no site do stackoverflow
- OWNER_ID: ??
- CREATION_DATE: data de criação da questão.

**SO_TAG** - *Stack Overflow Tag*:
- ID: indetificador da tag na base de dados
- SO_ID: indetificador da tag no site do stackoverflow

**SO_QUESTION_TAG** - Tabela de relacionamento NxN de questões e tags
- ID: identificador do relacionamento
- Q_ID: identificador da questão na base de dados
- TAG_ID: identificador da tag na base de dados


# Tratamento dos Dados, formato nó e link.

In [2]:
conn = sqlite3.connect('dados/grafo.db', timeout=10)
cursor_grafo = conn.cursor()

cursor_grafo.execute('''DROP TABLE NODE''');
cursor_grafo.execute('''CREATE TABLE NODE
             ([ID] INTEGER PRIMARY KEY,[NAME] text, [SIZE] integer)''')

cursor_grafo.execute('''DROP TABLE LINK''');
cursor_grafo.execute('''CREATE TABLE LINK
             ([ID] INTEGER PRIMARY KEY,[SOURCE] text, [TARGET] text, [SIZE] integer)''')

conn.commit()

In [ ]:
# Insere Nós e Tamanho do nó.
# Percorrer tabela de Tags E fazer count na tabela de relacionamento
conn = sqlite3.connect('dados/grafo.db', timeout=10)
cursor_grafo = conn.cursor()

conn = sqlite3.connect('dados/tags_database.db', timeout=10)
cursor_so = conn.cursor()

# Select das tags
query = """SELECT * from SO_TAG"""
cursor_so.execute(query)
rows = cursor_so.fetchall()
total = len(rows)
print("Total rows are:  ", total)

import time

start_time_total = time.time()


max_id = 0;
i = 0
links_processados = 0;
for row in rows:
    
    tag_id = row[0]
    tag_so_id = row[1]
    #print("Id: ", tag_id)
    #print("SO_ID: ", tag_so_id) 
    query = """SELECT COUNT(ID) from SO_QUESTION_TAG WHERE TAG_ID = """ + str(tag_id);
    cursor_so.execute(query)    
    size = cursor_so.fetchone()[0];
    #query = """DELETE from SO_QUESTION_TAG WHERE TAG_ID = """ + str(tag_id);
    #cursor_so.execute(query)
    #size = cursor_so.rowcount;
    #print("Size: ", size)     
    links_processados = links_processados + size;
    if size > 10000:
        sql = ''' INSERT INTO NODE(ID,NAME,SIZE) VALUES(?,?,?) '''
        node = (tag_id, tag_so_id, size)
        cursor_grafo.execute(sql, node)
    
    if(i % 100 == 0):        
        start_time_lote = time.time()
        print("Progresso: ", i)
        print("--- %s seconds total ---" % (time.time() - start_time_total))
        print("--- %s seconds lote ---" % (time.time() - start_time_lote))
        if(i > 18000):
            break;
    i = i + 1
    #if i == 10:
    #    break;
cursor_grafo.close()
cursor_so.close()

Total rows are:   46431
Progresso:  0
--- 27.044936895370483 seconds total ---
--- 0.0014488697052001953 seconds lote ---
Progresso:  100
--- 155.43002939224243 seconds total ---
--- 0.00010061264038085938 seconds lote ---
Progresso:  200
--- 287.7208411693573 seconds total ---
--- 0.00011563301086425781 seconds lote ---
Progresso:  300
--- 442.2576913833618 seconds total ---
--- 0.00010514259338378906 seconds lote ---
Progresso:  400
--- 606.0511319637299 seconds total ---
--- 0.00016880035400390625 seconds lote ---
Progresso:  500
--- 772.8132600784302 seconds total ---
--- 0.00037479400634765625 seconds lote ---
Progresso:  600
--- 914.8979318141937 seconds total ---
--- 0.00040340423583984375 seconds lote ---
Progresso:  700
--- 1085.354008436203 seconds total ---
--- 0.00019311904907226562 seconds lote ---


In [12]:
cursor_grafo.close()
cursor_so.close()